# ***creation de graphiques d'analyses des résultats post-traitements***

In [2]:
import geopandas as gp
import pandas as pd
import altair as alt
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
from MMM import import_fichiers_mmm
import Connexion_Transfert as ct
import random

# 1.Analyse des sources de données

> ## 1.1 fichiers sources

In [44]:
resultatsLineaires=r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\FV_affectation_complete.shp'
mmm_simple,cle_mmm_rhv=import_fichiers_mmm(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\MMM\MMM_Maj_EDA2017\simplifie\MMM_simple_2019.shp',
                                            r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\appariementV0.shp')
pour_calcul=gp.read_file(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\pour_calcul.shp')
rdpt=gp.read_file(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\rdpoint_ext.shp')
cbs=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier Personnel de Martin SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\CBS2017\Filaire_global_metropole_DENSIFIER.shp')
traf_voies_1234=gp.read_file(r'C:\Users\martin.schoreisz\Box\Dossier Personnel de Martin SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp')

> ## 1.2. Comptages  
Analyse des points de comptages : calcul de la longueur des tronçons RHV supportant un type_cpt "permanent" ou "ponctuel", par classe de RHV, par rapport à la longueur totale des tronçons de cette classe

In [4]:
#import des résultats filaires
res=gp.read_file(resultatsLineaires)

In [47]:
#preparation des données
res=res.loc[~res.cat_rhv.isna()].copy()
res['long']=res.geometry.length/1000
res['cat_rhv_simple']=res.cat_rhv.apply(lambda x : x[-1])
res['grp_indic']=res.apply(lambda x : 'comptage' if x['type_cpt'] in ('permanent','ponctuel') else 'autre', axis=1)

In [6]:
#analyse du pourcentage de longueur ayant fait l'objet de comptage
baseComptageParCatRhv=alt.Chart(res.groupby(['cat_rhv_simple','grp_indic']).long.sum().reset_index()).mark_bar().encode(
    x=alt.X('cat_rhv_simple', title='Catégorie RHV simplifiée'),
    color=alt.Color('grp_indic',title='Source des trafics'))
chartComptageParCatRhv=baseComptageParCatRhv.encode(y=alt.Y('long', title='longueur (km)')) | baseComptageParCatRhv.encode(
    y=alt.Y('long', title='longueur (km)', stack='normalize', axis=alt.Axis(format='%')))
chartComptageParCatRhv

alt.HConcatChart(...)

> ## 1.3. MMM  
Comparaison des données de trafics des voies du MMM avec celle issues des points de comptages, par catégorie du RHV. Il faut pour ça enlever les ronds points du RHV

In [7]:
#rond_points
listIdentRdpt=gp.sjoin(res,rdpt,op='within').ident.tolist()

In [8]:
#preparer les donées pour compraraison : enlver la Rocade et autres 2*2 voies de la comparaison et calcul des écarts puis on ne garde que le valeur max du MMM pour chaque ligne si doublons
CompMMMCpt=cle_mmm_rhv[['NO','ident']].merge(pour_calcul.loc[(pour_calcul.nom_sect.isin(['intra-bld', 'extra-Rocade_rd', 'extra-Rocade-Sud_rg',
       'extra-Rocade-NO_rg', 'intra-Rocade-Sud_rg', 'Bastide','intra-Rocade-NO_rg', 'intra-Rocade_rd', 'rocade bretelle','Bld RG'])) & 
        (~pour_calcul.ident.isin(listIdentRdpt)) & (pour_calcul.type_cpt.isin(('permanent','ponctuel')))][['ident','cat_rhv_si','tmjo_2_sen','nom_sect', 'numero']], 
                                             on='ident').merge(mmm_simple[['NO','tmja_tv']])
CompMMMCpt.drop_duplicates(['tmjo_2_sen','tmja_tv'], inplace=True)
CompMMMCpt['ecart']=CompMMMCpt.tmjo_2_sen-CompMMMCpt.tmja_tv
CompMMMCpt['ecart_cpt']=CompMMMCpt.tmjo_2_sen/CompMMMCpt.tmja_tv
CompMMMCpt=CompMMMCpt.loc[CompMMMCpt.groupby(['tmjo_2_sen']).tmja_tv.transform(max)==CompMMMCpt.tmja_tv].copy()

In [30]:
#calcul des graphs
chartDensiteProbabiliteEcart=alt.Chart(CompMMMCpt.loc[(CompMMMCpt.ecart_cpt<12)],width=325, height=220).transform_density(
    'ecart_cpt',groupby=['cat_rhv_si'],    as_=['ecart_cpt', 'density'],
).mark_area().encode(
    x=alt.X("ecart_cpt:Q",title= 'rapport tmjo mesuré / tmja MMM'),
    y=alt.Y('density:Q', title='densité de probablité'),
    facet=alt.Facet('cat_rhv_si:N', columns=1, title=None))

chartQuantileEcart=alt.vconcat(*[
(alt.Chart(CompMMMCpt.loc[(CompMMMCpt.ecart_cpt<12) & (CompMMMCpt.cat_rhv_si==str(i))], title=f'catégorie RHV : {str(i)}').transform_quantile(
    'ecart_cpt', step=0.01,groupby=['cat_rhv_si']
).mark_line().encode(
    x=alt.X('prob:Q', axis=alt.Axis(tickMinStep=0.05)) ,
    y=alt.Y('value:Q', scale=alt.Scale(domain=[0,12]))) + 
alt.Chart(pd.DataFrame({'x':[e/10 for e in range(0,11)], 'y':[0.6]*11, 'y2':[1.3]*11})).mark_area(opacity=0.3, color='green').encode(
    x=alt.X('x', title = 'quantile'), 
    y=alt.Y('y', title= 'rapport tmjo mesuré / tmja MMM'), y2='y2') 
).properties(width=300, height=200) for i in range(1,4)])

chartExampleVariation=alt.Chart(CompMMMCpt.loc[(0.3<CompMMMCpt.ecart_cpt) | (2<CompMMMCpt.ecart_cpt) & (CompMMMCpt.ecart_cpt<12)],width=300, height=450).mark_circle().encode(
    x=alt.X('tmjo_2_sen', title='tmjo mesuré'), 
    y=alt.Y('tmja_tv', title='tmja MMM'),
    tooltip=['numero','ident'],
    color='numero',
    text='numero',
    facet=alt.Facet('cat_rhv_si:N', columns=3, title=None)).resolve_scale(
    color='independent', y='independent', x='independent')

In [19]:
alt.hconcat(chartQuantileEcart,chartDensiteProbabiliteEcart)

alt.HConcatChart(...)

In [31]:
chartExampleVariation

alt.Chart(...)

>## 1.4 fichiers CBS

In [97]:
#calcul tmja_tv
cbs['tmjatv']=cbs.apply(lambda x : ((float(x.MT.replace(',','.'))+float(x.PT.replace(',','.')))*12) + 
                        ((float(x.ME.replace(',','.'))+float(x.PE.replace(',','.')))*4) + 
                        ((float(x.MT.replace(',','.'))+float(x.PT.replace(',','.')))*8), axis=1)
#limitation du fichier resultat aux données de comptage

In [98]:
#export si besoin
#cbs.to_file(r'C:\Users\martin.schoreisz\Box\Dossier Personnel de Martin SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\CBS2017\cbs_tmja.shp')

In [99]:
#transfert de données par buffer puis concatenation
trafic_comptage=res.loc[res['grp_indic']=='comptage'].merge(traf_voies_1234[['ident','idtronc']], on='ident')
trafic_comptage['buffer']=trafic_comptage.geometry.buffer(8)
trafic_comptage=trafic_comptage.set_geometry('buffer')
idtroncBuff=trafic_comptage[['buffer','idtronc']].dissolve(by='idtronc').reset_index()
jointureCbsTrafCpt=gp.sjoin(cbs,idtroncBuff.to_crs('epsg:3945'), op='within')#.set_geometry('geometry_left')
jointureCbsTrafCpt[['CAT_DIG', 'IDENT_1', 'NAME', 'NOM', 'MT', 'ME', 'MN', 'PT', 'PE', 'PN',
       'SSCR_AW', 'VPKWD', 'VPKWE', 'VPKWN', 'VLKWD', 'VLKWE', 'VLKWN',
       'STRONR', 'STEIG', 'FLOW', 'NVOIE', 'ID', 'HA', 'HA_ATT', 'SSCR_AW_L',
       'SSCR_AW_R', 'geometry', 'tmjatv', 'index_right', 'idtronc']].to_file(r'C:\Users\martin.schoreisz\Box\Dossier Personnel de Martin SCHOREISZ\Linearisation_BM\C19SA0101\Doc_travail\Donnees_produites\Donnees\compCbsTraCpt.shp')

In [109]:
#somme des trafics CBS par idtronc
cbsTmja2Sens=jointureCbsTrafCpt[['geometry','idtronc','tmjatv']].dissolve(by='idtronc', aggfunc='sum').reset_index()
#jointure avec les données mesurées
compCbsTraf=cbsTmja2Sens.merge(traf_voies_1234[['ident','idtronc','tmjo_2_sen']], on='idtronc')

In [110]:
compCbsTraf

,idtronc,geometry,tmjatv,ident,tmjo_2_sen
0,2.0,"LINESTRING Z (1421342.610 4184937.620 0.000, 1...",14583.2,19653,6774.0
1,4.0,"LINESTRING Z (1421245.500 4185360.440 0.000, 1...",3516.8,19654,3252.0
2,5.0,MULTILINESTRING Z ((1413556.900 4182943.440 0....,80336.0,20736,31090.0
3,5.0,MULTILINESTRING Z ((1413556.900 4182943.440 0....,80336.0,24208,31090.0
4,6.0,"LINESTRING Z (1411258.850 4191375.930 0.000, 1...",10699.6,37774,9716.0
...,...,...,...,...,...
6809,2792311.0,MULTILINESTRING Z ((1421366.850 4193302.650 0....,31076.0,5441,3503.0
6810,2792311.0,MULTILINESTRING Z ((1421366.850 4193302.650 0....,31076.0,54649,3503.0
6811,2792311.0,MULTILINESTRING Z ((1421366.850 4193302.650 0....,31076.0,54650,3503.0
6812,2792320.0,"LINESTRING Z (1409086.490 4192560.070 0.000, 1...",9496.0,52081,5730.0
